# BEAT-PD Challenge

Challenge website : https://www.synapse.org/#!Synapse:syn20825169/wiki/596118

Data information : https://www.synapse.org/#!Synapse:syn20825169/wiki/600405


### Ideas/Doubts [Laureano]

VAD like thing to remove unwanted data?
modified MFCC?
X,Y,Z = relative positions or acceleration?

Imp: Predict per person. Maybe UBM like thing and adapt it

In [1]:
# Import required libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML, display

# Imports for the high pass signal
from scipy.signal import butter, freqz, lfilter

# KFold
from sklearn.model_selection import KFold

# Import required modules
from sklearn.preprocessing import StandardScaler

import os.path

# To write WAV File
from scipy.io.wavfile import write

# To make derivative work on multiple CPUs
from concurrent.futures import ProcessPoolExecutor
from functools import partial

import sys

from sklearn.model_selection import StratifiedKFold

%load_ext autoreload
%autoreload 2

In [2]:
# import transform_data
from transform_data import *
from create_graphs import *


In [3]:
# Data paths
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"

# FIXME : Move this to data?
path_save_accelerometer_plots = "/home/sjoshi/codes/python/BeatPD/code/accelerometer_plots/"

possible to have participant characteristics from additional db data? ex https://ieeexplore.ieee.org/abstract/document/7911257

# CIS-PD Database

Contains 16 subject_id (patients) for the training set 

- Gender: 11 Male, 5 Female 
- Race: 15 White, 1 NA
- Ethnicity: 15 Not Hispanic or Latino, 1 Unknown
- Age average (standard deviation) : 62.8125 (10.8579)

### CIS-PD: Basic accelerometer visualization

In [ ]:
data_type = "cis"

# TODO: explain
path_train_data, df_train_label = define_data_type(data_type=data_type, data_dir=data_dir)
# display(df_train_label)
# List of interesting measurement id we want to look at
# list_measurement_id=[#'ab5287f4-8261-47ad-8ff2-22b5fe5d246e',
#'db2e053a-0fb8-4206-891a-6f079fb14e3a']#,
# 'ef5b1267-c212-46c5-aab0-4f4437bc6c67',
# '4ec74fb9-7347-435d-83dc-79ad74c3bc49',
# '8e8539ad-8841-476b-b15c-888ce3461989',
# '22b88456-fe8f-4138-af55-be12afca4b81',
# 'ad84583d-e5ae-4926-b077-531a0f7d08a9',
# 'eef56825-940a-4c3e-aebb-60838d60869e',
# 'e0441156-c4b8-467c-8f4f-3b532d594d8f',
# '464ac314-6c4b-4c4a-957c-28a2339150d6']

list_measurement_id = [
    "5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a",
    "cc7b822c-e310-46f0-a8ea-98c95fdb67a1",
    "5163afe8-a6b0-4ea4-b2ba-9b4501dd5912",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "2e3a4c9c-ff01-4a28-bfcf-ce9b7633a39d",  # no inactivity should be removed
    "3cf49c01-0499-4bad-9167-67691711204a",  # no inactivity should be removed PAS LA??
    "3d0f965c-9d72-43d1-9369-1ea3acf963cc",  # PAS LA ???
    "4b269cc2-8f0c-4816-adbf-10c0069b8833",
    "4bc51b90-bfce-4231-85e1-5de3b4bc0745",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
]  # bit of inactivity in the middle]

list_measurement_id = [
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
]

# Filter df_train_label according to the measurement_id we are most interested in
df_train_label = interesting_patients(
    df_train_label=df_train_label, list_measurement_id=list_measurement_id
)

# Display filtered df_train_label
display(df_train_label)

# path_no_inactivity_data = remove_inactivity_pct_change(df_train_label)

# Plot the accelerometer data
plot_accelerometer(df_train_label,
                   data_type=data_type,
                   path_train_data=path_train_data,
                   path_accelerometer_plots=path_save_accelerometer_plots
)

### CIS-PD: Create Masks for inactivity removal 

In [ ]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)

print(path_train_data)

In [1]:
data_type = "cis"

remove_inactivity_highpass(
    df_train_label,
    path_train_data,
    data_type,
    energy_threshold=5,
    duration_threshold=3000,
    plot_frequency_response=False,
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.'+
    data_subset+'.high_pass_mask/')

NameError: name 'remove_inactivity_highpass' is not defined

### CIS-PD: Create first derivative

In [2]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)


do_work = partial(
    get_first_derivative, 
    path_train_data=path_train_data,
    derivative_path=data_dir+"cis-pd.training_data.derivative_original_data/",
    padding=True, 
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.training_data.high_pass_mask/',
)

num_jobs = 1
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

NameError: name 'define_data_type' is not defined

### CIS-PD: Create WAV files - Training Data

#### For Y

In [ ]:
data_type = "cis"
data_subset='training_data'

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)
do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Y/',
    sAxis='Y',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 8
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

#### CIS-PD Write Wav Ancillary Data Y

In [4]:
data_type = "cis"
data_subset='ancillary_data' 

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)
do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Y/',
    sAxis='Y',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 6
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/10005cf2-85cf-47ad-8441-f15fb1509f69.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/f25cfad6-1835-4f78-9192-d28581ffa4d8.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/10005cf2-85cf-47ad-8441-f15fb1509f69.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e9fa3c88-3eba-45a6-93c7-3ef4e5c8f846.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/f55eac42-c808-447b-9b9b-02c10cee579f.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/541740be-1158-4169-becb-5ca4926ecd0c.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5cf1a9ff-b13c-457b-a661-ccd617842027.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/78428c75-bb3f-48c9-87f1-

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/406135de-3156-4f03-b38d-7758bf8d58bb.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1bb07775-5199-4fca-90e7-28038a3dd8f1.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/27770f06-a85b-483c-a793-60f7c5f5167e.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/42ebe42f-ce33-4633-aa65-575472ff0c99.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/860cd080-5f6f-45ac-a1cc-f791b13e6555.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5a9b01ef-2deb-4a98-96fe-5797822334fe.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/c34dc766-74a2-46b4-82a3-a218303ffb26.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1bb07775-5199-4fca-90e7

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/39c50dd4-4433-4ead-9c97-fddfb96974f7.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fd46a3a4-965f-4e8e-a957-7aea9fa8d3de.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/593174b5-8802-41e5-84bf-713da62667b9.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/d3a1eb8d-e993-4b41-9a28-2f1205d0e08c.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/18ae03ad-e265-4d50-bd50-6baad42007fb.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/aa4a923e-d509-4abd-9c0a-4a6c637e37de.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/593174b5-8802-41e5-84bf-713da62667b9.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fd46a3a4-965f-4e8e-

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/4b9d0dc6-e291-4a8f-a838-750038e789cf.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e2b029c2-af71-406c-8232-116cf1293ecd.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/46b51fb7-b901-43cd-b5f8-d1c9aef22e37.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/c7b935d5-11e3-4ddd-a6f5-e2d507cdee9b.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a7015551-83fe-453c-9452-6d7fef9a2a26.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/eab28778-e4db-462d-baf2-30ebdcd2ad7e.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/c014e4c6-68b5-48ba-b817-2317b93f8ea4.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a7015551-83fe-453c-94

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/96f949fa-febc-4175-b654-9973e68bb408.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2e905c74-b895-4b0d-a6ca-9274f8cee767.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/0bb8a566-fbf5-4710-a4cd-2524a2a328e3.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fba2201b-50bb-4f64-a4a2-71735ce1a589.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a0d4fa63-827d-4c97-affd-bfa4e2dce425.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a29a468d-1db0-4a3a-a677-f81786fbef87.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/96f949fa-febc-4175-b654-9973e68bb408.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/b3e3dce2-5c37-4d09-8

...

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/170303da-330c-44dd-88df-d8c59148bfb2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1a3b8150-bfb4-4063-a084-55252ee3c458.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/377bc4a0-8da1-49f9-8a2d-283a02f571bc.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/85179480-bbc9-4241-8d08-4b8a8fb644f2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/786296cd-872e-431f-a851-8c7691d0d176.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/96428e26-10ed-421e-b853-8eccd5a07961.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/170303da-330c-44dd-88df-d8c59148bfb2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/85179480-bbc9-4241-

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e92bcd74-81cf-4a0c-b11f-c2f7c5647315.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/aaab722d-32e2-4fda-a428-9e8365074d84.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/bf67f14e-5ecc-4b46-94b9-c242a5e7d7d1.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/8cb01d26-2ab0-4ed6-a686-41b2bd4a00c5.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/f4db1429-1632-45fa-b5d2-571e1c79aaf7.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/6fb8093d-5b26-4580-a4f8-0c7ded523f01.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/9be6f67c-4a95-4814-bf28-b0c7d1921012.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/d79e42ba-0a21-4afd-9

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fee641d0-3c0f-4de4-88e8-a9512eb228ac.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/41a599c3-3325-4f3d-87cc-e5e63fa7ff5e.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/93b9c431-e7a9-48e5-aea6-0b05f1b08eea.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2d073ae5-33f0-412a-b526-0ab42337355c.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/26636484-5c37-4e59-9158-5ff11c58dcae.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/fee641d0-3c0f-4de4-88e8-a9512eb228ac.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/7a267ca8-d47d-4932-bec6-3ae53f68a7aa.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/55753704-38d4-46a7-b

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/1286c4d8-5e44-4ede-aabc-7b8662e2156a.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/e1586f1b-d51b-4468-b21e-d951c0b3fa29.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2eb0b592-b038-4c15-931d-b59cc97a8975.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/74485ef6-0db6-4b06-8941-f55f624fe569.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/074c5992-3ef8-4187-a895-f8650f8e245f.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/57c3696d-eaec-4d34-b8ae-2addf6d6fa78.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/2eb0b592-b038-4c15-931d-b59cc97a8975.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/61afcf16-a7c5-490d-9562

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/132b839a-5383-446f-8120-a439e18cae47.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5d0ea615-e03f-4d1c-9772-e7e83e5f40ef.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/db80e345-ad48-43d6-b7a9-4d2518e9831d.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/310a15c9-1863-4524-904e-2dc730d58329.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/a1612a58-a5e1-43bb-8f31-0cbd97a5d806.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/63c0aeb7-0d28-4dd4-b80b-58e158b99016.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/edcb47d7-33d9-4fe3-be2d-9afd08b90fc5.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.ancillary_data.wav_Y/5d0ea615-e03f-4d1c-977

#### For Z

In [ ]:
data_type = "cis"
data_subset='training_data'

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset) 

do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Z/',
    sAxis='Z',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 8
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

#### CIS-PD Write Wav Ancillary Data Z

In [ ]:
data_type = "cis"
data_subset='ancillary_data' 

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)

do_work = partial(
    write_wav,
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Z/',
    sAxis='Z',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 6
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

## KFold 

In [8]:
def get_k_fold(df_train_label,
               n_splits=5,
               subject_id=None,
               data_subset='training_data',
               data_real_subtype=""):
    """
    Function that returns a list of X dataframes (X is according to the number of n_splits chosen)

    The dataframes are the labels needed according to the split 

    Keyword Arguments:
    - df_train_label: Dataframe containing the labels
    - n_split: Optional. The number of folds. Default: 5
    - subject_id: Optional. Specify a subject_id to get measurement_id only for that subject_id 
    - data_real_subtype: Only for REAL-PD database
    """
    kf = StratifiedKFold(n_splits) # shuffle??? 

    # Building the dataframe to split
    X = []

    # if we want the data split for one specific subject_id
    if subject_id:
        df_train_label_subject_id = df_train_label.groupby("subject_id")
        X = df_train_label_subject_id.get_group(subject_id)
    else:
        # if we want to have all a split for all data no matter the subject_id
        # NOTE: I didn't make sure to have one subject_id represented in both train/test
        for idx in df_train_label.index:
            X.append([df_train_label["measurement_id"][idx]])
        X = pd.DataFrame(X)

    kf.get_n_splits(X)

    # Building lists of df_train_label because we have by default 5 splits,
    # so the lists will contain 5 DataFrames with different split indices required
    list_df_train_label = list()
    list_df_test_label = list()
    split_idx = 0
    
    # Removing NaN values only for the purpose of kf.split as stratified kfold don't like NaN values
    print(type(X))
    print('Before contains NaN ? : ', X.isnull().sum())
    X_no_nan = prepro_missing_values(X)
    print('After contains NaN ? : ', X_no_nan.isnull().sum())
    
    if subject_id == 1046:
        # 1046 is balanced on tremor because that's the only labels we have for that patient
        y = X_no_nan.iloc[:,-1]
    else:
        # We do the KFOLDs on a balanced on/off only 
        y = X_no_nan.iloc[:,-3] 

    print("----- " + str(subject_id) + " -----")
    for train_index, test_index in kf.split(X_no_nan, y):
        print(len(X))
        print(len(df_train_label))
        df_train_label = X.iloc[train_index]
        df_test_label = X.iloc[test_index]
        
        # Following is just to see if the splits are balanced or not 
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        train_0, train_1, train_2, train_3, train_4 = len(train_y[train_y==0]), \
                                                        len(train_y[train_y==1]), \
                                                        len(train_y[train_y==2]), \
                                                        len(train_y[train_y==3]), \
                                                        len(train_y[train_y==4])
        test_0, test_1, test_2, test_3, test_4  = len(test_y[test_y==0]), \
                                                  len(test_y[test_y==1]), \
                                                  len(test_y[test_y==2]), \
                                                  len(test_y[test_y==3]), \
                                                  len(test_y[test_y==4])
        print('>Train: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d Test: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d' % (train_0, train_1, train_2, train_3, train_4, \
                                                        test_0, test_1, test_2, test_3, test_4))
        
        list_df_train_label.append(df_train_label)
        list_df_test_label.append(df_test_label)

        # name of the file according to its database and type
        # NOTE: Be careful that the end of the name of the folder where to save the kfolds is hardcoded here
        path_save_k_fold_dataframes = (
            data_dir + data_type + "-pd."+data_subset+".k_fold_v3/" + data_real_subtype + "/"
        )
        df_train_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_train_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        df_test_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_test_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        split_idx = split_idx + 1
    return list_df_train_label, list_df_test_label

### Create the K-Fold files for the CIS database 

In [ ]:
# Define the data type as we have two databases
data_type = "cis"
path_train_data, df_train_label = define_data_type(data_type=data_type)

# Group data by subject_id
df_train_label_subject_id = df_train_label.groupby("subject_id")

data_subset = 'training_data'

# Go through the subject_id and k-fold their data
# FIXME: get_k_fold could me renamed to just create the folds, save them, not return anything
for subject_id, value in df_train_label_subject_id:
    list_df_train_label, list_df_test_label = get_k_fold(
        df_train_label=df_train_label,
        n_splits=5,
        training_or_ancillary=data_subset,
        subject_id=subject_id
    )


### Create the K-Fold Files for the REAL database

In [ ]:
# To generate the files, you have to uncomment one data_real_subtype at a time and
# execute this cell 3 times for the 3 subtypes.

data_type = "real"
data_subset='training_data' # Either training_data or ancillary_data 

for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    
    path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset,
                                                   data_real_subtype)

    # Group data by subject_id
    df_train_label_subject_id = df_train_label.groupby("subject_id")

    # Go through the subject_id and k-fold their data
    for subject_id, value in df_train_label_subject_id:
        list_df_train_label, list_df_test_label = get_k_fold(
            df_train_label=df_train_label,
            n_splits=5,
            subject_id=subject_id,
            training_or_ancillary=data_subset,
            data_real_subtype=data_real_subtype,
        )

# REAL-PD Database

This database, originally named "Parkinson@Home" is renamed to "Real-PD" for this challenge. The study was made over 2 weeks, with at home monitoring. 

The devices used are an android phone, a motorolla watch. 
- `smartwatch_accelerometer` and `smartwatch_gyroscope` : Motorolla Watch
- `smartphone_accelerometer` : Android phone 

-> Question: so is the smartwatch & smartphone accelerometer should measure the same movements? 

The REAL-PD database has many missing values. 

The subject_id `hbv013` is the only one without missing data. Other patients all have at least one missing symptom (`diskenisia`, ) or two (`on/off and tremor`, `on_off and dyskinesia`, `dyskinesia and tremor`) missing.

Measurements id with no data (`on_off`, `dyskinesia` and `tremor` are all missing):
- `b50d1b0c-2cd1-45f8-9097-0742e5cbbcc8`
- `b598c177-4e38-4ea8-8543-bd8f7e580f96`
- `cf841bf8-0082-4ea3-999f-1f43e39a8dc6`
- `b1e15f8a-109f-459b-ba87-46899240ee66`
- `6f0e2580-56ec-4743-9356-d3e4d9a0aee5`
- `773536f6-9b70-43d0-b099-5d167d74924a`
- `54a0e841-ad45-4ba7-ac83-1785c5f7748b`
- `cd9ed2e2-7e04-44c7-b041-7788f133c193`
- `a6954a91-338b-4523-9e4a-5e69a8fac206`

The 3 symptoms are reported as follows in this dataset: 
- `on_off = {0,1}`
  - `Off` : 0 (medication is wearing off) 
  - `On` : 1 (medication is working)
  
- `dyskinesia = {0,1,2}`
  - Without dyskinesia: 0 
  - Non-troublesome dyskinesia: 1 
  - Severe dyskinesia: 2 
  
- `tremor = {0,1,2,3,4}` 
The description of the database mentions `tremor` is rated from 0 to 4 according to its severity, but from all the data, the maximum value of `tremor` recorded is 3. 

Data:
- ancillary
- clinical : UPDRS evaluation score 
- demographics : #TODO


### REAL-PD: Create Masks for inactivity removal for all subtypes

In [ ]:
data_type = "real"
#data_real_subtype='smartphone_accelerometer'
#data_real_subtype = "smartwatch_accelerometer"
#data_real_subtype='smartwatch_gyroscope'

# List of interesting measurement id we want to look at
# list_measurement_id = [
#     "5b4c7c81-659d-40ea-a1fd-59622074fd10",
#     "ee053d95-c155-400d-ae42-fe24834ad4a9",
#     "ce51ee31-8553-4321-9f83-8cd3dabe2f66",
#     "e07708ff-7b8d-4070-af70-3aa81423ab5b",
#     #'7d3f4b7a-167f-4a26-9062-94ce9d8794c1',
#     "99af8d14-cd09-4107-9502-355378ba4e08",
#     #'7d5ac31a-cb53-40f7-8188-0b13724ea55c',
#     "9e43840b-dd89-498b-af1a-a62896a4d5d9",
#     "e391f546-bf8a-46c7-a16c-95bc02f40629",
# ]

# list_measurement_id = ['0c310608-1a32-4b09-b164-375d93ddb2aa']

# Filter df_train_label according to the measurement_id we are most interested in
# df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    data_subset='ancillary_data' #training_data
    print(data_real_subtype)
    path_train_data, df_train_label = define_data_type(data_type,
                                                       data_dir,
                                                       data_subset,
                                                       data_real_subtype)

    list_measurement_id = ['f465145e-562d-4c02-b347-bc42ae3b4998']#['33f5a031-43a8-496a-89ee-0b9d99019617', '2515fa8d-8320-4010-bcd9-5dba52b1b5ba']
    # Filter df_train_label according to the measurement_id we are most interested in
    df_train_label = interesting_patients(df_train_label=df_train_label,
                                          list_measurement_id=list_measurement_id)

    remove_inactivity_highpass(
        df_train_label,
        path_train_data=path_train_data,
        data_type=data_type,
        energy_threshold=5,
        duration_threshold=3000,
        plot_frequency_response=False,
        plot_accelerometer_after_removal=False,
        mask_path=data_dir+'/real-pd.'+
        data_subset+
        '.high_pass_mask/'+data_real_subtype+'/')

### REAL-PD: Create first derivative for all subtypes 

In [ ]:
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    path_train_data, df_train_label = define_data_type(data_type=data_type)
    print(len(df_train_label))
    for idx in df_train_label.index:
        try:
#             print('where we get the file : ', path_train_data)
            df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
        except FileNotFoundError:
            print('Removing ' + df_train_label["measurement_id"][idx] +
                  ' as it doesn\'t exist for ' +
                  data_real_subtype)
            df_train_label = df_train_label.drop(idx)
        print(len(df_train_label))
    do_work = partial(
        get_first_derivative, 
        path_train_data=path_train_data,
        derivative_path="real-pd.training_data.derivative_original_data/"+data_real_subtype+'/',
        padding=True, 
        mask_path=data_dir+'/real-pd.training_data.high_pass_mask/'+data_real_subtype+'/'
    )

    num_jobs = 8
    with ProcessPoolExecutor(num_jobs) as ex:
        results = list(ex.map(do_work, df_train_label['measurement_id']))


### REAL-PD: Create WAV files for all subtypes

### X Axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="X", data_type="real")

# data_type ="real"
# #FIXME add the one missing when it's done running 
# #'smartphone_accelerometer', 'smartwatch_accelerometer', smartwatch_gyroscope
# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset = 'ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type, data_subset, data_real_subtype)
# #     list_mesurement_id=['33f5a031-43a8-496a-89ee-0b9d99019617']
#     # Filter df_train_label according to the measurement_id we are most interested in
# #     df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)

#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
        
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'real-pd.'+data_subset+'.wav_X/'+data_real_subtype+'/',
#         sAxis='X',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/'
#     )

#     num_jobs = 6
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

### Y axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Y", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data'#'training_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)

#     do_work = partial(
#         write_wav,
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Y/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Y',
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

Removing 7fde941c-2d8a-4b33-b32d-af016161ee1e as it doesn't exist for smartphone_accelerometer
Removing 92eea111-b6b0-4d07-a624-cf2b0ecea769 as it doesn't exist for smartphone_accelerometer
Removing 3f5adb13-512c-4090-96ac-707703077845 as it doesn't exist for smartphone_accelerometer
Removing 539ab11e-0d9f-4d42-a209-5261a3655275 as it doesn't exist for smartphone_accelerometer
Removing 475759e6-bdad-44f4-a1be-63d18e475352 as it doesn't exist for smartphone_accelerometer
Removing 6542d311-9d13-400a-8a3c-61f0d52ef6f5 as it doesn't exist for smartphone_accelerometer
Removing 41c392cb-3495-4138-a1b2-f056704f160c as it doesn't exist for smartphone_accelerometer
Removing eb5c7874-8961-4bac-b1cb-e35e4d715ca5 as it doesn't exist for smartphone_accelerometer
Removing 0b37bc64-905b-4234-af29-9760c9e16346 as it doesn't exist for smartphone_accelerometer
Removing c519cfef-d785-4382-91e7-545824817cf0 as it doesn't exist for smartphone_accelerometer
Removing caeaea08-2477-4b5b-8d57-cf85043459c7 as i

Removing 537b5264-ea4c-4539-924a-4bf6e9eb69ba as it doesn't exist for smartphone_accelerometer
Removing cf49a3ae-8921-4264-99ee-a06d5f05575a as it doesn't exist for smartphone_accelerometer
Removing 99200aa0-4834-4f34-96d9-71c3c1b765b1 as it doesn't exist for smartphone_accelerometer
Removing f2a6831f-e712-4eb7-b150-ac44f520679c as it doesn't exist for smartphone_accelerometer
Removing fd3e05f6-8d9a-43c9-8a93-2231d341650d as it doesn't exist for smartphone_accelerometer
Removing 732aa52a-ce30-43d7-9fb0-647aeeb49957 as it doesn't exist for smartphone_accelerometer
Removing b09b5104-3a37-4819-b7f5-ba721d024722 as it doesn't exist for smartphone_accelerometer
Removing c0a5ca86-ac24-4d73-8623-c2923677c617 as it doesn't exist for smartphone_accelerometer
Removing f0911246-5bbd-4299-9ead-a60551b2ebfc as it doesn't exist for smartphone_accelerometer
Removing 18cd67ad-b280-4520-9582-d38b4f8004f8 as it doesn't exist for smartphone_accelerometer
Removing 59a2afdb-b324-421d-a0ba-e1b08dcd0585 as i

File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/7d9e47b4-75dc-4721-befc-c0a7fc1f68f4.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/4e26e20d-344b-4bbf-9dbf-4a389c73784d.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/58ea7790-a93c-4a9f-81c0-4757f237346e.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/ed80c2a5-135e-4f90-9649-b5a6db5aa399.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/a4bf0da9-7116-4b57-ada2-d4e6fa116253.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/7d9e47b4-75dc-4721-befc-c0a7fc1f68f4.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.anci

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/9dda3848-0758-4889-81ac-7a2d4fee50ef.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/caf3bb66-3d7e-47c2-8437-8d7a28d3d13b.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/d02209e2-ccb6-43ca-88a4-5053c51312f9.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/7e5db015-a0ad-4898-af48-8dbb927aad28.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/9410e6a1-be87-4b03-be0d-fae812c52d91.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/caf3bb66-3d7e-47c2-8437-8d7a28d3d13b.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancill

File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/757b411a-6b2a-42fb-a806-4d89bd7d6fa2.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/7fcb1c8d-9dd8-4940-b54e-6ecd5082e110.wav
File exist :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/d417f853-2455-4038-9086-fe7832ff9010.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/27b2058e-874e-4392-91b3-cde65928908f.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/13811568-56c3-4f62-9188-6f34fd972b03.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/a368d2c7-1bbe-413e-a463-dcc1735f3b16.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancill

...

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/8bda2c9c-a6fc-46b2-9c20-38581fca9f35.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/a396ee3f-de01-4ed7-bf0d-f6a89263e4fd.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/a396ee3f-de01-4ed7-bf0d-f6a89263e4fd.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/a2ac41d0-c8ef-469c-987e-36754259a829.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/a2ac41d0-c8ef-469c-987e-36754259a829.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/9376a3b9-cf26-4aad-ad4c-89533c641921.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/Be

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/f09a7000-8e42-44b7-a1e4-9806819c32b5.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/a1ccefe7-0c6e-4b11-a9f1-60328ca87900.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/a1ccefe7-0c6e-4b11-a9f1-60328ca87900.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/05e2a329-cb22-400b-bb73-7fa68c670cd2.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/05e2a329-cb22-400b-bb73-7fa68c670cd2.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/752c9ff8-2f18-4694-b2ff-537b0241e349.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/Be

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/c9ef48a5-f40e-4e78-aa24-63c12d8b2698.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/d38d71da-d767-4168-82c3-1741327fe6bb.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/d38d71da-d767-4168-82c3-1741327fe6bb.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/b6f4e0d2-4710-4691-b123-787d59ebf910.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/b6f4e0d2-4710-4691-b123-787d59ebf910.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/622887f2-5081-4bd1-a6cd-348660e7d84f.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/B

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/b976f06b-0cc7-4f52-a671-16a6b4b0a799.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/9682b198-4951-4489-8c90-95d4b2d83069.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/9682b198-4951-4489-8c90-95d4b2d83069.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/4d62ffb3-6352-46d0-b1b3-17e06d962fc0.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/4d62ffb3-6352-46d0-b1b3-17e06d962fc0.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/25e1e50b-af9c-47d0-b3e2-734fc9836d56.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/Be

PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/1872cee9-3d53-4e8d-8d95-2145dd4e436e.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/d678d036-c5cf-43e6-ae7b-c63eecbc2c1b.wav
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/b288f078-92bf-4907-ab75-6e594b92ecaf.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/d678d036-c5cf-43e6-ae7b-c63eecbc2c1b.csv
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.high_pass_mask/smartphone_accelerometer/b288f078-92bf-4907-ab75-6e594b92ecaf.csv
File path :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//real-pd.ancillary_data.wav_Y/smartphone_accelerometer/ab6d059d-4437-4c10-96e8-8445cad60813.wav
PATH !!! :  /home/sjoshi/codes/python/BeatPD/data/Be

### Z Axis

In [11]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Z", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartwatch_gyroscope']:#['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     display(df_train_label)
#     for idx in df_train_label.index:
#         try:
#             print(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
#     display(df_train_label)
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Z/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Z'
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))
        

data_type :  real
aaaaaaaaaaaaaa
path train data :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/
path_train_labels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.data_labels/REAL-PD_Ancillary_Data_IDs_Labels.csv


,measurement_id,subject_id,on_off,dyskinesia,tremor
0,2515fa8d-8320-4010-bcd9-5dba52b1b5ba,hbv058,0.0,0.0,NaN
1,7fde941c-2d8a-4b33-b32d-af016161ee1e,hbv079,NaN,NaN,NaN
2,92eea111-b6b0-4d07-a624-cf2b0ecea769,hbv079,NaN,NaN,NaN
3,3f5adb13-512c-4090-96ac-707703077845,hbv090,NaN,NaN,NaN
4,42d34bdc-54a9-4b4f-bed6-c02219c6061f,hbv058,0.0,0.0,NaN
5,3bc6b0d1-0204-4a3d-827a-1579a76b411e,hbv065,NaN,NaN,0.0
6,803d4ad6-328d-494f-8923-329175c8b07f,hbv065,0.0,0.0,0.0
7,f6dde26f-279c-4d83-9e5e-8b25cb7e31bd,hbv002,NaN,NaN,NaN
8,ed9d2a14-cb01-42e6-85f0-180b70b2f821,hbv024,1.0,0.0,0.0
9,83cdbcb3-034a-4eb2-9cc6-3757dd4aff22,hbv065,0.0,0.0,0.0


/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/2515fa8d-8320-4010-bcd9-5dba52b1b5ba.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/7fde941c-2d8a-4b33-b32d-af016161ee1e.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/92eea111-b6b0-4d07-a624-cf2b0ecea769.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/3f5adb13-512c-4090-96ac-707703077845.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/42d34bdc-54a9-4b4f-bed6-c02219c6061f.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/3bc6b0d1-0204-4a3d-827a-1579a76b411e.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/803d4ad6-328d-494f-8923-329175c8b07f.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/f

/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/15a5b255-d6ea-455c-9a67-02ac71d3890e.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/4b8e9621-5dca-4511-a40f-980dec857c57.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/caeaea08-2477-4b5b-8d57-cf85043459c7.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/80a009f0-1461-4dfb-b735-7748ba78d3f9.csv
Removing 80a009f0-1461-4dfb-b735-7748ba78d3f9 as it doesn't exist for smartwatch_gyroscope
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/e8a35db7-db21-4a1f-a278-caad9cf481fb.csv
Removing e8a35db7-db21-4a1f-a278-caad9cf481fb as it doesn't exist for smartwatch_gyroscope
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/4c5e949d-4813-496d-b4f6-bb445f8a9646.csv
/home/sjoshi/codes/python/BeatPD/data/

/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/73585d45-ca5a-4aa1-b6fb-3c777aad24c9.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/7480fb4c-76e9-426e-87fc-73799bb63203.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/46f2b5c1-ada6-4483-9681-29f8eb1f6d16.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/a4b30e09-e663-4634-9d8e-2bcb733c4f04.csv
Removing a4b30e09-e663-4634-9d8e-2bcb733c4f04 as it doesn't exist for smartwatch_gyroscope
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/9b7fb594-b20a-47fa-a579-159dc2dbeb9f.csv
Removing 9b7fb594-b20a-47fa-a579-159dc2dbeb9f as it doesn't exist for smartwatch_gyroscope
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/7066484a-e9f8-4100-b9b7-78533368566a.csv
/home/sjoshi/codes/python/BeatPD/data/

...

/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/732aa52a-ce30-43d7-9fb0-647aeeb49957.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/6b83a350-00a7-4a54-831d-80a88d8d8d93.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/b09b5104-3a37-4819-b7f5-ba721d024722.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/c0a5ca86-ac24-4d73-8623-c2923677c617.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/1322e18a-897f-4a5a-a0bf-6e12d7a3a54b.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/0b6fe925-fd2d-406e-8cf8-299e3e948792.csv
Removing 0b6fe925-fd2d-406e-8cf8-299e3e948792 as it doesn't exist for smartwatch_gyroscope
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/f0911246-5bbd-4299-9ead-a60551b2ebfc.csv

/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/3c3141f1-eaa1-45f7-a761-d869cda10322.csv
/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.ancillary_data/smartwatch_gyroscope/bc0bd952-89cf-456d-ba66-656808a1c6be.csv


,measurement_id,subject_id,on_off,dyskinesia,tremor
0,2515fa8d-8320-4010-bcd9-5dba52b1b5ba,hbv058,0.0,0.0,NaN
1,7fde941c-2d8a-4b33-b32d-af016161ee1e,hbv079,NaN,NaN,NaN
2,92eea111-b6b0-4d07-a624-cf2b0ecea769,hbv079,NaN,NaN,NaN
3,3f5adb13-512c-4090-96ac-707703077845,hbv090,NaN,NaN,NaN
4,42d34bdc-54a9-4b4f-bed6-c02219c6061f,hbv058,0.0,0.0,NaN
5,3bc6b0d1-0204-4a3d-827a-1579a76b411e,hbv065,NaN,NaN,0.0
6,803d4ad6-328d-494f-8923-329175c8b07f,hbv065,0.0,0.0,0.0
7,f6dde26f-279c-4d83-9e5e-8b25cb7e31bd,hbv002,NaN,NaN,NaN
8,ed9d2a14-cb01-42e6-85f0-180b70b2f821,hbv024,1.0,0.0,0.0
9,83cdbcb3-034a-4eb2-9cc6-3757dd4aff22,hbv065,0.0,0.0,0.0


Process ForkProcess-5:
Process ForkProcess-1:
Process ForkProcess-3:
Process ForkProcess-4:
Process ForkProcess-2:
Process ForkProcess-7:
Process ForkProcess-8:
Traceback (most recent call last):
  File "/home/mpgill/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/mpgill/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mpgill/anaconda3/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/mpgill/anaconda3/lib/python3.7/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/home/mpgill/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/mpgill/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/mpgill/an

KeyboardInterrupt: 

# Tests & Drafts, back-up space that's not important, just notes

In [ ]:
# Change the default option to display all row with display(DF)
# pd.set_option('display.max_rows', 1000)

### Tried to do convolution instead of np.multiply and dot product to get the derivative

In [ ]:
df2 = pd.DataFrame(
    np.array(
        [
            [0.2, 1, 5, 9],
            [0.4, 2, 6, 10],
            [0.6, 3, 7, 11],
            [0.8, 4, 8, 12],
            [1, 5, 9, 13],
            [1.2, 6, 10, 14],
            [1.4, 7, 11, 15],
            [1.6, 7, 11, 15],
            [1.8, 8, 12, 16],
            [2, 9, 13, 16],
        ]
    ),
    columns=["Timestamp", "X", "Y", "Z"],
)
display(df2)
m = np.linspace(-3, 3, num=2 * 3 + 1)
display(m)

np.convolve(df2.loc[0:6, "X"], m)

### How to filter a DataFrame

In [ ]:
# Analysis of the data to find edge cases


# Create variable with TRUE if nationality is USA
dys = df_train_label["dyskinesia"] > 1

# Create variable with TRUE if age is greater than 50
tre = df_train_label["on_off"] > 0

# Select all cases where nationality is USA and age is greater than 50
df_train_label[dys & tre]

In [ ]:
def remove_inactivity_max(df_train_label):
    last_filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    display(last_filtered_value)
    for idx in df_train_label.index:
        df_allo = []
        df_train_data = pd.read_csv(
            path_train_data + df_train_label["measurement_id"][idx] + ".csv"
        )

        # Get the absolute max value for X, Y, Z
        max_values = df_train_data.iloc[:, -3:].abs().max()

        # Compute what is 5% of that max
        thresold_energy = 5
        df_treshold = (max_values * thresold_energy) / 100

        # display(df_train_data)
        # Candidates are the frames where X, Y, Z are below that treshold (5% of the max)
        #         df_candidates = df_train_data[(df_train_data.X.abs() <= df_treshold.X) &
        #                                      (df_train_data.Y.abs() <= df_treshold.Y) &
        #                                      (df_train_data.Z.abs() <= df_treshold.Z)]
        # display(df_candidates)
        for idx2 in df_train_data.index:
            # print('df_train_data[idx2]')
            # display(df_train_data.iloc[idx2,-3:])
            last_filtered_value = filtered_value
            filtered_value = last_filtered_value + 0.004 * (
                df_train_data.iloc[idx2, -3:] - last_filtered_value
            )
            y = pd.DataFrame(columns=["Timestamp"])
            y = pd.concat(
                [y, pd.DataFrame([df_train_data.iloc[idx2, 0]], columns=["Timestamp"])],
                ignore_index=True,
            )
            #             print('display y :')
            #             display(y)
            #             print('end display y')

            #             print('display filtered value')
            #             display(pd.DataFrame(filtered_value).transpose())
            #             print('end display filtered value')
            df_allo.append(
                pd.concat([y, pd.DataFrame(filtered_value).transpose()], axis=1)
            )
        #             print('display df_allo')
        #             display(df_allo)

        # FIXME : change the name df_allo
        df_allo = pd.DataFrame(df_allo, columns=("Timestamp", "X", "Y", "Z"))

        df_allo.plot(x="Timestamp", legend=True, subplots=True, title="allo")
        stop()


#         v_candidate_x = pd.DataFrame({'Candidate':list(0)})
#         v_candidate_x = np.where(df_train_data.X.abs() <= df_treshold.X, 1, 0)

In [ ]:
# df_zeros = pd.DataFrame([False,True,False,False,False,True,False]).astype(int)

df_zeros = np.array([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=bool)

display(df_zeros.astype(int))
count = 0
duration_threshold = 2
indices_list = []  # List of tuples
howmany = 0
for i in range(0, len(df_zeros)):
    if df_zeros[i] == 1:
        count = count + 1
        print("1 à lindex", i)
    else:
        if count >= duration_threshold:
            print("threshold atteint start ", start, " end at ", end)
            start = i - count
            end = i - 1
            indices_list.append((start, end))
            howmany = howmany + 1
            count = 0
        # if it doesn't reach the threshold, we change the 1 for 0 because we don't want to remove those
        elif count >= 1:
            print("Effacer les 1 de ", start, " a ", end)
            df_zeros[i - count : i] = [0] * (end - start)
            count = 0

display(df_zeros.astype(int))